In [ ]:
"""
This is the Wang and Buszaki inteerneuron model implementation in Brian Simulator. The code is the example code given in the 
Brian Docs website. The model is stimulated using the frozen noise protocol. 
The input file used has a sampling rate of 20KHz and the simulation is run for 1 second. 
"""

In [ ]:
from brian2 import *
import matplotlib.pyplot as plt
import scipy.io as mat
import numpy as np

In [ ]:
I_theory = mat.loadmat("C:\\Users\\parih\\OneDrive\\Documents\\GitHub\\Microcircuit_Model\\input_inhibitory_tau_50_sampling_20_kHz.mat")["input_theory_i"]
hidden_state = mat.loadmat("C:\\Users\\parih\\OneDrive\\Documents\\GitHub\\Microcircuit_Model\\input_inhibitory_tau_50_sampling_20_kHz.mat")["hidden_state_i"]
I_scale = 2 #scaling factor for the input current - amplitude of the input current
I_baseline = 0.3 #baseline current - mean of the input current 
I_theory = I_theory.flatten()
Isyn_val = I_scale*I_theory+I_baseline
dt = 0.05*ms #timestep = 1/sampling rate - should stay consistent over the entire simulation
simtime = 1000*ms
Iapp = TimedArray(Isyn_val*uA, dt=dt)

In [ ]:
mean = np.mean(I_theory)
amplitude = np.max(I_theory) - np.min(I_theory)
print(mean,amplitude)

In [ ]:
#parameters
Cm = 1*uF 
iL = 0.1*msiemens
vL = -65*mV
vNa = 55*mV
vK = -90*mV
iNa = 35*msiemens
iK = 9*msiemens

#interneuron equations
eqs = '''
dv_int/dt = (-Il - INa - Ik + I)/Cm : volt
Il = iL * (v_int - vL) : amp
INa = iNa * m**3 * h * (v_int - vNa) : amp
Ik = iK * n**4 * (v_int - vK) : amp
m = alpham / (alpham + betam) : 1
alpham = -0.1*(v_int/mV+35)/(exp(-0.1*(v_int/mV+35))-1)/ms : Hz
betam = 4*exp(-(v_int/mV+60)/18)/ms : Hz
dh/dt = 5 * (alphah * (1-h) - betah * h) : 1
alphah = 0.07*exp(-(v_int/mV+58)/20)/ms : Hz
betah = 1/(1+exp(-0.1*(v_int/mV+28)))/ms : Hz
dn/dt = 5 * (alphan * (1-n) - betan * n) : 1
alphan = -0.01*(v_int/mV+34)/(exp(-0.1*(v_int/mV+34))-1)/ms : Hz
betan = 0.125*exp(-(v_int/mV+44)/80)/ms : Hz

I = Iapp(t) : amp
'''
#create neuron groups
neuron = NeuronGroup(1, eqs, threshold='v_int > -20*mV', method='euler', refractory=2 * ms,)
neuron.v_int = -70*mV
neuron.h = 1
neuron.n = 0

#set monitors
M = StateMonitor(neuron, 'v_int', record=True) # or record=0.1*ms
I = StateMonitor(neuron, 'I', record=True)
S = SpikeMonitor(neuron)

run(simtime)

In [ ]:
subplot(311)
plot(M.t/ms, M[0].v_int/mV)
xlabel('Time (ms)')
ylabel('Membrane Potential (mV)')
plt.title('Wang and Buzsaki Interneuron Model')
subplot(312)
plot(I.t/ms, hidden_state[0:size(I.t)], color = 'black')
xlabel('Time(ms)')
ylabel('hidden state ')
subplot(313)	
plot(I.t/ms, Isyn_val[:len(I.t/ms)]) 
xlabel('Time(ms)')
ylabel('Input Current (nA)')


In [ ]:
'firing rate = ' + str(size(S.t)/5000)